In [1]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 952.5 kB/s eta 0:00:001m820.8 kB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 2.6 MB/s eta 0:00:003.0 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.9/423.9 kB 5.1 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 5.0 MB/s eta 0:00:00m eta 0:00:010:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 5.4 MB/s eta 0:00:00


In [4]:
!pip install python-dotenv

In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [17]:
#print(os.environ['OPENAI_API_KEY'])

In [3]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-4"
else:
    llm_model = "gpt-3.5-turbo-0301"

In [15]:
def get_completion(prompt, model=llm_model):
    messages = [{"role": "user", "content": prompt}]
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0, 
    )
    print(response)
    return response.choices[0].message.content


In [16]:
get_completion("Hello?")

ChatCompletion(id='chatcmpl-9qMnp74z3XMpWcrpTvPy2BGQaCTZR', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello! How can I assist you today?', role='assistant', function_call=None, tool_calls=None))], created=1722267737, model='gpt-4-0613', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=9, prompt_tokens=9, total_tokens=18))


'Hello! How can I assist you today?'

In [18]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [19]:
style = """American English \
in a calm and respectful tone
"""

In [20]:
prompt = f"""Translate the text \
that is delimited by triple backticks 
into a style that is {style}.
text: ```{customer_email}```
"""

print(prompt)

Translate the text that is delimited by triple backticks 
into a style that is American English in a calm and respectful tone
.
text: ```
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
```



In [21]:
response = get_completion(prompt)

ChatCompletion(id='chatcmpl-9qN4ky8Tuhhy3o7SUzNk3N0lwOTmh', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="```\nI'm really upset that my blender lid came off and splattered my kitchen walls with smoothie! To make matters worse, the warranty doesn't cover the cost of cleaning up my kitchen. I need your help right now, friend!\n```", role='assistant', function_call=None, tool_calls=None))], created=1722268786, model='gpt-4-0613', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=50, prompt_tokens=89, total_tokens=139))


In [22]:
response

"```\nI'm really upset that my blender lid came off and splattered my kitchen walls with smoothie! To make matters worse, the warranty doesn't cover the cost of cleaning up my kitchen. I need your help right now, friend!\n```"

## Chat API : LangChain

Let's try how we can do the same using LangChain.

In [32]:
#!pip install langchain_community
#!pip install --upgrade langchain


In [33]:
from langchain.chat_models import ChatOpenAI

In [35]:
chat = ChatOpenAI(temperature=0.0, model=llm_model)
#chat

In [36]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

In [37]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)


In [39]:
prompt_template

ChatPromptTemplate(input_variables=['style', 'text'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['style', 'text'], template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n'))])

In [38]:
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['style', 'text'], template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n')

In [40]:
customer_style = """American English \
in a calm and respectful tone
"""

In [41]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [42]:
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)

In [43]:
print(type(customer_messages))
print(type(customer_messages[0]))

<class 'list'>
<class 'langchain_core.messages.human.HumanMessage'>


In [44]:
print(customer_messages[0])

content="Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone\n. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\n"


In [45]:
# Call the LLM to translate to the style of the customer message
customer_response = chat(customer_messages)

/Users/c5367983/anaconda3/envs/LangChain-Application/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [46]:
print(customer_response.content)

I'm quite upset that my blender lid came off and splattered my kitchen walls with smoothie. To add to the frustration, the warranty doesn't cover the cost of cleaning up my kitchen. I need your help immediately, please.


In [47]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

In [48]:
service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""

In [49]:
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

print(service_messages[0].content)

Translate the text that is delimited by triple backticks into a style that is a polite tone that speaks in English Pirate. text: ```Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!
```



In [50]:
service_response = chat(service_messages)
print(service_response.content)

```Ahoy there, me hearty, the warranty be not coverin' the cost of swabbin' yer galley, for 'tis ye who be at fault for mishandlin' yer mixin' contraption by forgettin' to secure the top 'fore settin' it to whirl. Hard cheese, matey! Until we cross paths again!```


## Output Parsers

Let's start with defining how we would like the LLM output to look like:

In [51]:
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}


{'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable!'}

In [53]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [61]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n'))]


In [62]:
messages = prompt_template.format_messages(text=customer_review)
chat = ChatOpenAI(temperature=0.0, model=llm_model)
response = chat(messages)
print(response.content)

{
"gift": true,
"delivery_days": 2,
"price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
}


In [63]:
type(response.content)

str

In [65]:
# You will get an error by running this line of code 
# because'gift' is not a dictionary
# 'gift' is a string
#response.content.get('gift')

### Parse the LLM output string into a Python dictionary

In [66]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [67]:
gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]

In [68]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [69]:
format_instructions = output_parser.get_format_instructions()

In [70]:
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.
	"delivery_days": string  // How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.
	"price_value": string  // Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.
}
```


In [71]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review, 
                                format_instructions=format_instructions)

In [72]:
print(messages[0].content)

For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.

text: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```

In [73]:
response = chat(messages)

In [74]:
print(response.content)

```json
{
	"gift": "True",
	"delivery_days": "2",
	"price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
}
```


In [75]:
type(response)

langchain_core.messages.ai.AIMessage

In [76]:
output_dict = output_parser.parse(response.content)

In [77]:
output_dict

{'gift': 'True',
 'delivery_days': '2',
 'price_value': ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]}

In [78]:
output_dict.get('delivery_days')

'2'

In [60]:
# from openai import AzureOpenAI

# client = AzureOpenAI(
#     api_key = "4c81e9b0eae04972a9132ea399d6746a",# leave your key here
#     api_version =  "2023-07-01-preview",
#     azure_endpoint = "https://ai-proxy.lab.epam.com"
#   )

# deployment_name = "gpt-35-turbo"
  
#   # Please only use this one if you absolutely need it. It's slower and more expensive.
#   # deployment_name = "gpt-4"
#   # deployment_name = "gpt-4-32k"

#   # For embeddings only, but small private models may perform better and cheaper
#   # https://huggingface.co/spaces/mteb/leaderboard
#   # deployment_name = "text-embedding-ada-002"
  

# print(client.chat.completions.create(
#     model=deployment_name,
#     temperature=0,
#     messages=[
#         {
#           "role": "user",
#           "content": "how are you?",
#         },
#     ],
#   ))
